# process_cmip6_files.ipynb
Post-processing of CMIP6 data that have been downloaded from ESGF:
1. Regrid to regular lon/lat grid (using CDO).

B. S. Grandey, 2022

In [1]:
! date

Thu Jan 27 19:40:06 +08 2022


In [2]:
import contextlib
import pathlib
from cdo import Cdo, CDOException

In [3]:
cdo = Cdo()

print(f'CDO version: {cdo.version()}')
print(f'cdo.py bindings version: {cdo.__version__()}')

CDO version: 2.0.3
cdo.py bindings version: 1.5.4


## Base paths

In [4]:
# CMIP6 input data downloaded from ESGF (organised as <variable>/<source_id>_<member_id>/<instance_id>)
in_base = pathlib.Path('~/Data/p22b/CMIP6/').expanduser()
# Output data
out_base = pathlib.Path('~/Data/p22c/CMIP6/').expanduser()
out_base.mkdir(exist_ok=True)

## Function regrid available data (using CDO) for a given variable, table_id, source-member pair, and experiment
Notes regarding available interpolation methods, after testing CDO's gencon2, gencon, and genbic on Omon zos data:
1. For unstructured grids, gencon/gencon2 works but not genbic ("Bilinear/bicubic interpolation doesn't support unstructured source grids!").
2. For curvilinear grids, genbic works. gencon/gencon2 mostly works, but fails for some grids (with most common error being "Source grid cell corner coordinates missing!").
3. For lonlat grids, all methods work.

Based on the above observations, one possibility would be to try gencon2 (2nd order conservative). If gencon2 fails, then use gencon (1st order conservative). If gencon2 and gencon both fail, then use genbic (bicubic). In case all three fail, include gennn (nearest neighbour) as a final fallback.

In [5]:
def regrid_using_cdo(variable='zos',
                     table_id='Omon',
                     source_member='ACCESS-CM2_r1i1p1f1',
                     experiment='historical',
                     preferred_methods=['gencon2', 'gencon', 'genbic', 'gennn'],  # highest priority first
                     force=False,  # overwrite previously produced files?
                    ):
    print(f'---- {variable} {table_id} {source_member} {experiment} ({variable}/{source_member}/) ----')
    # List of output files written
    out_list = []
    # Search for available input directories, then identify most appropriate directory
    in_dirs = list(in_base.glob(f'{variable}/{source_member}/CMIP6.*.{experiment}.*.{table_id}.{variable}.*'))
    in_dirs = sorted(in_dirs)
    # Case: no input directories
    if len(in_dirs) == 0:
        print('No suitable input data found')
        return None
    # Case: one suitable input directory
    elif len(in_dirs) == 1:
        in_dir = in_dirs[0]
        print(f'1 suitable input dir: {in_dir.name} ({len(list(in_dir.glob("*.nc")))} nc file(s))')
    # Case: more than one suitable input directory
    else:
        print(f'{len(in_dirs)} suitable input dirs:')
        for d in in_dirs:
            print(f'{d.name} ({len(list(d.glob("*.nc")))} nc file(s))')
        # Select most recently dated version
        vmax = 0  # initialise "max" version as 0
        for d in in_dirs:
            vmax = max(vmax, int(d.name[-8:]))
        for d in in_dirs:
            if str(vmax) not in d.name:
                in_dirs.remove(d)
        # After version selection, does only one input dir remain?
        if len(in_dirs) == 1:
            in_dir = in_dirs[0]
            print(f'Using {in_dir.name} (most recent) ')
        # Prefer data that has already been remapped
        else:
            for d in in_dirs:
                if '.gn.' in d.name:
                    in_dirs.remove(d)
            in_dir = in_dirs[0]
            print(f'Using {in_dir.name} (non-native grid)')
    # Output data directory
    out_dir = out_base.joinpath(f'01_regrid/{variable}/{source_member}/{in_dir.name}')
    out_dir.mkdir(parents=True, exist_ok=True)
    # Identify input data files
    in_fns = sorted(list(in_dir.glob('*.nc')))
    # Input grid type
    grid_str = [s for s in cdo.sinfo(input=f'{in_fns[0]}') if 'points' in s][0]
    in_grid_type = grid_str.split(':')[1].strip()
    in_grid_points = grid_str.split(':')[-1].strip()
    print(f'Input data grid type is {in_grid_type} ({in_grid_points})')
    # If points=1 (e.g. due to already being a global mean), then no need to regrid
    if in_grid_points == 'points=1':
        print('No need to regrid')
        for in_fn in in_fns:
            out_fn = out_dir.joinpath(f'{in_fn.stem}.1d.nc')
            if out_fn.exists() and not force:  # do not overwrite (unless force=True)
                print(f'{out_fn.name} already exists')
                continue
            cdo.selname(variable, input=f'{in_fn}', output=f'{out_fn}')
            if out_fn.exists():
                print(f'Written {out_fn.name}')
                out_list.append(out_fn.name)
            else:
                print(f'Failed to write {out_fn.name}')
        return out_list
    # Generate regridding weights file
    for method in preferred_methods:
        print(f'Trying {method}')
        weights_fn = out_dir.joinpath(f'weights.{method}.nc')
        # For unstructured grids, do not use gencon2
        if method == 'gencon2' and in_grid_type == 'unstructured':
            print(f'Skipping gencon2 for unstructured grid')
            continue
        # If weights file already exists, use pre-existing file (unless force=True)
        if weights_fn.exists() and not force:
            print(f'{weights_fn.name} already exists')
            break  # if weights file exists, no need to try other methods
        # Otherwise, try calculating weights and writing file
        else:
            try:
                with contextlib.redirect_stdout(None):  # suppress CDO error message details
                    if method == 'gencon2':
                        cdo_res = cdo.gencon2('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')
                    elif method == 'gencon':
                        cdo_res = cdo.gencon('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')                    
                    elif method == 'genbic':
                        cdo_res = cdo.genbic('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')
                    elif method == 'gennn':
                        cdo_res = cdo.gennn('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')
                    else:
                        continue
                if weights_fn.exists():
                    print(f'Written {weights_fn.name}')
                    out_list.append(weights_fn.name)
                    break  # if file written successfully, then no need to try other methods
            except CDOException:
                print(f'Failed to write {weights_fn.name}')
    # Loop over input data files and regrid
    print(f'{len(in_fns)} input data file(s) to regrid')
    for in_fn in in_fns:
        # Output data filename
        out_fn = out_dir.joinpath(f'{in_fn.stem}.{method[3:]}.nc')
        # If output file already exists, do not overwrite (unless force=True)
        if out_fn.exists() and not force:
            print(f'{out_fn.name} already exists')
            continue
        # Regrid using CDO
        cdo_res = cdo.sellonlatbox(0, 360, -90, 90,  # shift longitudes
                                   input=f'-selname,{variable} -remap,global_1,{weights_fn} {in_fn}',
                                   output=f'{out_fn}')
        # Check if written
        if out_fn.exists():
            print(f'Written {out_fn.name}')
            out_list.append(out_fn.name)
        else:
            print(f'Failed to write {out_fn.name}')
    return out_list

## Apply function(s) to variables and experiments of interest

In [6]:
# For variables and experiments of interest, apply regridding function to all available source-member pairs
# Loop over variables of interest
for variable, table_id in [('zos', 'Omon'),]:
    # Identify and loop over available source-member pairs
    source_member_pairs = sorted([d.name for d in in_base.glob(f'{variable}/[!.]*_*')])
    for source_member in source_member_pairs:
        # Loop over experiments of interest
        for experiment in ['historical', ]:
            # Regrid
            temp = regrid_using_cdo(variable=variable,
                                    table_id=table_id,
                                    source_member=source_member,
                                    experiment=experiment,
                                    force=False)
        print('')  # empty line between source-member pairs

---- zos Omon ACCESS-CM2_r1i1p1f1 historical (zos/ACCESS-CM2_r1i1p1f1/) ----
1 suitable input dir: CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.r1i1p1f1.Omon.zos.gn.v20191108 (1 nc file(s))
Input data grid type is curvilinear (points=108000 (360x300))
Trying gencon2
Written weights.gencon2.nc
1 input data file(s) to regrid
Written zos_Omon_ACCESS-CM2_historical_r1i1p1f1_gn_185001-201412.con2.nc

---- zos Omon ACCESS-ESM1-5_r1i1p1f1 historical (zos/ACCESS-ESM1-5_r1i1p1f1/) ----
1 suitable input dir: CMIP6.CMIP.CSIRO.ACCESS-ESM1-5.historical.r1i1p1f1.Omon.zos.gn.v20191115 (1 nc file(s))
Input data grid type is curvilinear (points=108000 (360x300))
Trying gencon2
Written weights.gencon2.nc
1 input data file(s) to regrid
Written zos_Omon_ACCESS-ESM1-5_historical_r1i1p1f1_gn_185001-201412.con2.nc

---- zos Omon AWI-CM-1-1-MR_r1i1p1f1 historical (zos/AWI-CM-1-1-MR_r1i1p1f1/) ----
1 suitable input dir: CMIP6.CMIP.AWI.AWI-CM-1-1-MR.historical.r1i1p1f1.Omon.zos.gn.v20181218 (18 nc file(s))
Inp

Written weights.gencon2.nc
1 input data file(s) to regrid
Written zos_Omon_CanESM5_historical_r1i1p2f1_gn_185001-201412.con2.nc

---- zos Omon E3SM-1-1_r1i1p1f1 historical (zos/E3SM-1-1_r1i1p1f1/) ----
1 suitable input dir: CMIP6.CMIP.E3SM-Project.E3SM-1-1.historical.r1i1p1f1.Omon.zos.gr.v20191204 (33 nc file(s))
Input data grid type is lonlat (points=64800 (360x180))
Trying gencon2
Written weights.gencon2.nc
33 input data file(s) to regrid
Written zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_185001-185412.con2.nc
Written zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_185501-185912.con2.nc
Written zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_186001-186412.con2.nc
Written zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_186501-186912.con2.nc
Written zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_187001-187412.con2.nc
Written zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_187501-187912.con2.nc
Written zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_188001-188412.con2.nc
Written zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_188501-18

Written zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_191401-191412.con2.nc
Written zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_191501-191512.con2.nc
Written zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_191601-191612.con2.nc
Written zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_191701-191712.con2.nc
Written zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_191801-191812.con2.nc
Written zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_191901-191912.con2.nc
Written zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_192001-192012.con2.nc
Written zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_192101-192112.con2.nc
Written zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_192201-192212.con2.nc
Written zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_192301-192312.con2.nc
Written zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_192401-192412.con2.nc
Written zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_192501-192512.con2.nc
Written zos_Omon_EC-Earth3-AerChem_histo

Input data grid type is curvilinear (points=105704 (362x292))
Trying gencon2
Written weights.gencon2.nc
165 input data file(s) to regrid
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_185001-185012.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_185101-185112.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_185201-185212.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_185301-185312.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_185401-185412.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_185501-185512.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_185601-185612.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_185701-185712.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_185801-185812.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_185901-185912.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_186001-186012.con2.nc
Written zos_Omon_EC-Earth3-CC_historic

Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_195801-195812.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_195901-195912.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_196001-196012.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_196101-196112.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_196201-196212.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_196301-196312.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_196401-196412.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_196501-196512.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_196601-196612.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_196701-196712.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_196801-196812.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_196901-196912.con2.nc
Written zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_197001-197012.con2.nc
Written zos_Omon_EC-Earth

...

Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_187801-187812.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_187901-187912.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_188001-188012.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_188101-188112.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_188201-188212.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_188301-188312.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_188401-188412.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_188501-188512.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_188601-188612.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_188701-188712.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_188801-188812.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_188901-188912.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_189001-189012.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_189101-189112.

Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_199201-199212.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_199301-199312.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_199401-199412.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_199501-199512.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_199601-199612.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_199701-199712.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_199801-199812.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_199901-199912.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_200001-200012.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_200101-200112.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_200201-200212.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_200301-200312.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_200401-200412.con2.nc
Written zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_200501-200512.

Written zos_Omon_INM-CM5-0_historical_r1i1p1f1_gr1_190001-194912.con2.nc
Written zos_Omon_INM-CM5-0_historical_r1i1p1f1_gr1_195001-199912.con2.nc
Written zos_Omon_INM-CM5-0_historical_r1i1p1f1_gr1_200001-201412.con2.nc

---- zos Omon IPSL-CM6A-LR_r1i1p1f1 historical (zos/IPSL-CM6A-LR_r1i1p1f1/) ----
1 suitable input dir: CMIP6.CMIP.IPSL.IPSL-CM6A-LR.historical.r1i1p1f1.Omon.zos.gn.v20180803 (1 nc file(s))
Input data grid type is curvilinear (points=120184 (362x332))
Trying gencon2
Written weights.gencon2.nc
1 input data file(s) to regrid
Written zos_Omon_IPSL-CM6A-LR_historical_r1i1p1f1_gn_185001-201412.con2.nc

---- zos Omon KIOST-ESM_r1i1p1f1 historical (zos/KIOST-ESM_r1i1p1f1/) ----
2 suitable input dirs:
CMIP6.CMIP.KIOST.KIOST-ESM.historical.r1i1p1f1.Omon.zos.gr1.v20200825 (1 nc file(s))
CMIP6.CMIP.KIOST.KIOST-ESM.historical.r1i1p1f1.Omon.zos.gr1.v20210601 (1 nc file(s))
Using CMIP6.CMIP.KIOST.KIOST-ESM.historical.r1i1p1f1.Omon.zos.gr1.v20210601 (most recent) 
Input data grid type 

Input data grid type is curvilinear (points=138600 (360x385))
Trying gencon2
Written weights.gencon2.nc
17 input data file(s) to regrid
Written zos_Omon_NorESM2-MM_historical_r1i1p1f1_gn_185001-185912.con2.nc
Written zos_Omon_NorESM2-MM_historical_r1i1p1f1_gn_186001-186912.con2.nc
Written zos_Omon_NorESM2-MM_historical_r1i1p1f1_gn_187001-187912.con2.nc
Written zos_Omon_NorESM2-MM_historical_r1i1p1f1_gn_188001-188912.con2.nc
Written zos_Omon_NorESM2-MM_historical_r1i1p1f1_gn_189001-189912.con2.nc
Written zos_Omon_NorESM2-MM_historical_r1i1p1f1_gn_190001-190912.con2.nc
Written zos_Omon_NorESM2-MM_historical_r1i1p1f1_gn_191001-191912.con2.nc
Written zos_Omon_NorESM2-MM_historical_r1i1p1f1_gn_192001-192912.con2.nc
Written zos_Omon_NorESM2-MM_historical_r1i1p1f1_gn_193001-193912.con2.nc
Written zos_Omon_NorESM2-MM_historical_r1i1p1f1_gn_194001-194912.con2.nc
Written zos_Omon_NorESM2-MM_historical_r1i1p1f1_gn_195001-195912.con2.nc
Written zos_Omon_NorESM2-MM_historical_r1i1p1f1_gn_196001-196

Written zos_Omon_TaiESM1_historical_r1i1p1f1_gn_194301-194312.con2.nc
Written zos_Omon_TaiESM1_historical_r1i1p1f1_gn_194401-194412.con2.nc
Written zos_Omon_TaiESM1_historical_r1i1p1f1_gn_194501-194512.con2.nc
Written zos_Omon_TaiESM1_historical_r1i1p1f1_gn_194601-194612.con2.nc
Written zos_Omon_TaiESM1_historical_r1i1p1f1_gn_194701-194712.con2.nc
Written zos_Omon_TaiESM1_historical_r1i1p1f1_gn_194801-194812.con2.nc
Written zos_Omon_TaiESM1_historical_r1i1p1f1_gn_194901-194912.con2.nc
Written zos_Omon_TaiESM1_historical_r1i1p1f1_gn_195001-195012.con2.nc
Written zos_Omon_TaiESM1_historical_r1i1p1f1_gn_195101-195112.con2.nc
Written zos_Omon_TaiESM1_historical_r1i1p1f1_gn_195201-195212.con2.nc
Written zos_Omon_TaiESM1_historical_r1i1p1f1_gn_195301-195312.con2.nc
Written zos_Omon_TaiESM1_historical_r1i1p1f1_gn_195401-195412.con2.nc
Written zos_Omon_TaiESM1_historical_r1i1p1f1_gn_195501-195512.con2.nc
Written zos_Omon_TaiESM1_historical_r1i1p1f1_gn_195601-195612.con2.nc
Written zos_Omon_Tai

In [7]:
! date

Thu Jan 27 20:02:14 +08 2022
